# 9符合Python风格的对象
## 9.1对象表现形式
1. 对象字符串表示形式
    + repr()——对象的_\_repr__方法，以便于开发者理解的方式返回对象的字符串表示形式。
    + str()——对象的_\_str__方式，以便于用户理解的方式返回对象的字符串表示形式。
    + bytes()——对象的_\_bytes__方法，生成实例的二进制表达形式


In [3]:

from array import array
import math


class Vector2d:
    typecode = 'd'

    def __init__(self, x, y):
        self.x = float(x)
        self.y = float(y)

    def __iter__(self):
        """ 实现迭代方法，把vector2d变成可迭代对象 """
        return (i for i in (self.x, self.y))

    def __repr__(self):
        """ *self返回x,y变量给format，前提是实现__iter__方法 """
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(array(self.typecode, self)))

    def __eq__(self, other):
        return tuple(self) == tuple(other)

    def __abs__(self):
        return math.hypot(self.x, self.y)

    def __bool__(self):
        return bool(abs(self))

# BEGIN VECTOR2D_V1
    @classmethod  # <1>
    def frombytes(cls, octets):  # <2>
        typecode = chr(octets[0])  # <3>
        memv = memoryview(octets[1:]).cast(typecode)  # typecode=d，二进制转换成浮点型
        return cls(*memv)  # <5>
# END VECTOR2D_V1

In [4]:
v1 = Vector2d(3,4)
print(repr(v1))
v1_clone = eval(repr(v1))## 通过字符串形式克隆Vector2d实例
print(v1 == v1_clone)

Vector2d(3.0, 4.0)
True


In [7]:
print(bytes(v1))
v1_clone = Vector2d.frombytes(bytes(v1))## 通过字节形式克隆Vector2d实例
print(v1_clone == v1)

b'd\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\x10@'
True
